In [2]:
import numpy as np
import matplotlib.pyplot as plt
import openmc

In [55]:
NA = 6.022e23
s_per_year = 3.154e7

# Q1

In [190]:
# givens
flux = 1.2e12
m = 90 * 907.185 * 1e3 # tons to kg to g
x235 = 0.03

# xs
sig_c_u = 2.1e-24
sig_a_u = 2.3e-24
sig_a_pu = 600e-24

# half life
t_half = 24_400 * s_per_year
dc = np.log(2) / t_half

# use Eq. 5.89 for Pu
# atoms of U-238
m238 = m * (1 - x235)
M238 = openmc.data.atomic_mass("U238")
N238 = m238 / M238 * NA


# assume no Pu initially
# assume N238 is constant

# arrays
steps = 100000
time = s_per_year

dt = time / steps
nPus = np.zeros(steps)

# differencing
for i in range(steps):
    production = N238 * sig_c_u * flux
    if i == 0:
        nPus[i] = production * dt
        continue
    
    decay = dc * nPus[i - 1]
    absorption = sig_a_pu * flux * nPus[i - 1]

    nPus[i] = nPus[i - 1] + (production - decay - absorption) * dt

# output
MPu239 = openmc.data.atomic_mass("Pu239")
mPu = nPus[-1] / NA * MPu239

print(f"After 1 year, mass of Pu239 is {round(mPu * 1e-3, 5)} kg")

After 1 year, mass of Pu239 is 6.24982 kg


# Q2

In [123]:
# givens
fee = 1e-3 # $ / kWh(e)
cost = 253  # $ / kgU

bu = 33_000  # MWd/t
eta = .32

# converting fee
fee *= 24  # per kWd(e)
fee *= 1000  # per MWd(e)
fee *= eta  # per MWd

# converting bu
bu /= 1000  # t -> kg


# MWd/kg * $/MWd(e)
final = bu * fee
print(f"Disposal Cost: ${final}/kg")

Disposal Cost: $253.44/kg


# Q3

In [165]:
# givens
xi = 3e-2
xf = 0.8e-2

time = 3 * s_per_year  # years

# xs = total - elastic - inelastic - gamma
sig_235 = 400e-24

# differencing
dx = xi - xf
flux = dx / time / xi / sig_235

print(f"Flux: {round(flux * 1e-13, 3)}e13/cm^2/s")

Flux: 1.938e13/cm^2/s


# Q4

not in the real world. ava >= cap, only equal if plant runs at 100% the entire year.

# Q5

In [146]:
# givens
m = 87e3
mpb = m / 3  # mass per batch


# swu functions
def sep_pot(x):
    return (2 * x - 1) * np.log(x / (1 - x))


def swu(xp, xw=0.2/100, xf=0.71/100):
    p = 1
    f = (xp - xw) / (xf - xw)
    w = (xp - xf) / (xf - xw)

    vp = sep_pot(xp)
    vf = sep_pot(xf)
    vw = sep_pot(xw)

    return 1 * (p * vp + w * vw - f * vf)

## part a

In [170]:
xa1, xa2, xa3 = np.array([2.5, 2.9, 3.1]) / 100

swua1 = swu(xa1) * mpb
swua2 = swu(xa2) * mpb
swua3 = swu(xa3) * mpb

swua = swua1 + swua2 + swua3

def pprint(*args):
    for arg in args:
        print(round(arg * 1e-3, 3))
    return

pprint(swua1, swua2, swua3, swua)
#print(round(swua, 3))

93.778
118.725
131.43
343.934


## part b

In [171]:
xb1, xb2 = np.array([2.6, 3.2]) / 100

swub1 = swu(xb1) * mpb
swub2 = swu(xb2) * mpb
swub = (swub1 + swub2) / 2

pprint(swub1, swub2, swub)

99.95
137.831
118.891


## part c

In [172]:
swuc = swua + swub1 * 15 + swub2 * 14

pprint(swuc)

3772.827


# Q6

In [122]:
cf = .69

## part a

In [191]:
# givens
m = 95  # tonnes
P = 1200  # MWe
eta = .32

# calcs
bua = P / eta * cf * 365.24 / m
print(f"Burnup after 1 year: {round(bua, 3)} MWd/t")

Burnup after 1 year: 9947.984 MWd/t


## part b

What will be the average burnup, over the life of the plant, if 1/3 of the core is replaced every year? Assume a 30-year life for the plant and a constant capacity factor and efficiency.

In [194]:
N = 3

buf = 2 * N / (N + 1) * bua
buavg = (buf + bua) / 2

print(buf, buavg)

14921.976315789474 12434.980263157895
